In [136]:
import cv2
import os
import numpy as np

import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
from skimage.exposure import histogram
from matplotlib.pyplot import bar
from skimage.color import rgb2gray,rgb2hsv

# Convolution:
from scipy.signal import convolve2d
from scipy import fftpack
import math

from skimage.util import random_noise
from skimage.filters import median
from skimage.feature import canny


import matplotlib.pyplot as plt
import scipy

from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from scipy import signal
from scipy import misc # pip install Pillow
import matplotlib.pylab as pylab

from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.calibration import CalibratedClassifierCV

# DCT Functions

In [137]:
def dct2(a):
    return scipy.fftpack.dct( scipy.fftpack.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return scipy.fftpack.idct( scipy.fftpack.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

def zigzag(matrix,rows,columns):
    zigzagArray = []
    solution=[[] for i in range(rows+columns-1)] 
  
    for i in range(rows): 
        for j in range(columns): 
            sum=i+j 
            if(sum%2 ==0): 

                #add at beginning 
                solution[sum].insert(0,matrix[i][j]) 
            else: 

                #add at end of the list 
                solution[sum].append(matrix[i][j])
    for i in solution: 
        for j in i:
            zigzagArray.append(j) 
          
    return zigzagArray

# Fiddle with Images

In [138]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show() 
    
def Crop_Image(img):
    image=Image.open(img)
    image.load()

    image_data = np.asarray(image)
    image_data_bw = image_data.max(axis=2)
    non_empty_columns = np.where(image_data_bw.max(axis=0)>0)[0]
    non_empty_rows = np.where(image_data_bw.max(axis=1)>0)[0]
    cropBox = (min(non_empty_rows), max(non_empty_rows), min(non_empty_columns), max(non_empty_columns))

    image_data_new = image_data[cropBox[0]:cropBox[1]+1, cropBox[2]:cropBox[3]+1 , :]

    new_image = Image.fromarray(image_data_new)
    return new_image



def Reformat_Image(ImageFilePath):

    from PIL import Image
    image = Image.open(ImageFilePath, 'r')
    image_size = image.size
    width = image_size[0]
    height = image_size[1]


    background = Image.new('RGB', (32, 32), (255, 255, 255))
    offset = (int(round(((32 - width) / 2), 0)), int(round(((32 - height) / 2),0)))

    background.paste(image, offset)
    background = np.asarray(background) 
    return background[:,:,0]

# Get Your Data

In [139]:
def Extract_Images(path):
    Images = []
    labels = []
    th = 128
    for file_name in os.listdir(path):
        for pic in os.listdir(path+"/"+file_name):
                resizedImg = Reformat_Image(path+"/"+file_name+"/"+pic)
                binarizedImg = (resizedImg > th)
                #npImg = npImg.flatten()
                Images.append(binarizedImg)
                labels.append(file_name)
                
    Images = np.asarray(Images)
    labels = np.asarray(labels)
    return Images,labels


def Extract_Features_DCT(Images):
    #show_images([Images[0]],["whatever"])
    Images_Features = []
    for Img_Index in range(Images.shape[0]):
            img = Images[Img_Index]
            imgDCT = dct2(img)
            rows,columns = imgDCT.shape
            array = zigzag(imgDCT,rows,columns)
            feature = array[:400:1]
            Images_Features.append(feature)
    Images_Features = np.asarray(Images_Features)
    return Images_Features


def Extract_Features_Img(Images):
    Images_Features = []
    for Img_Index in range(Images.shape[0]):
            img = Images[Img_Index]
            feature = img.flatten()
            Images_Features.append(feature)
    Images_Features = np.asarray(Images_Features)
    return Images_Features

# Write the File

In [193]:
def Get_Labels(path):
    labels = []
    for file_name in os.listdir(path):
        labels.append(str(file_name))
    labels = np.asarray(labels)
    return labels
        
def Write_File(name,accuracies,labels):
    accuracies = np.asarray(accuracies)
    with open(name+".txt","w", encoding='utf-8') as file:
        for index_of_letter in range(accuracies.shape[0]):
            max_value = np.amax(accuracies[index_of_letter])
            index_of_max_value = np.where(accuracies[index_of_letter] == max_value)[0][0] # GET THE INDEX OF THE MAXIMUM ELEMNT
            written_letter = labels[index_of_max_value]
            file.write(written_letter)
            
        
    

# CLassifier

In [194]:
h = .02  # step size in the mesh

data_set_labels = Get_Labels("Letters")

names = [#"Nearest Neighbors", "Linear SVM", "RBF SVM",
             "Neural Net"]#, "Naive Bayes"]

classifiers = [
       # KNeighborsClassifier(3),
       # SVC(kernel="linear", C=0.025),
       # SVC(gamma=2, C=1),
        MLPClassifier(alpha=1, max_iter=1000)]
       # , GaussianNB()]


Images,Labels = Extract_Images("Letters")


Features = Extract_Features_DCT(Images)
X,y = Features,Labels


    
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=.2, random_state=42)

# iterate over classifiers
for name, clf in zip(names, classifiers):
    clf = CalibratedClassifierCV(clf)
    print("training classifier",name)
    clf.fit(X, y)
    score = clf.score(X_test, y_test)
    Write_File(name,clf.predict_proba(X_test),data_set_labels)
    print("Score of classifier",score)


(31,)
training classifier Neural Net
[0.00366551 0.00204477 0.00394677 0.00462453 0.00587388 0.00304731
 0.00351694 0.00458231 0.00277127 0.00319758 0.0019384  0.00321091
 0.00279078 0.00344702 0.00477402 0.00400545 0.00235456 0.00293441
 0.00252225 0.00400239 0.00155025 0.008193   0.00627559 0.00380845
 0.87701306 0.00375115 0.00334205 0.01400139 0.00565178 0.00263236
 0.00452983]
0.8770130606968488
[0.00212541 0.00206317 0.00407002 0.00457618 0.00979728 0.00373715
 0.00350075 0.00458031 0.00277891 0.00323055 0.00201167 0.00319862
 0.00268665 0.00345346 0.00480278 0.00399754 0.00228892 0.00293989
 0.00248988 0.00401929 0.00159084 0.00831272 0.00629247 0.0038341
 0.87600797 0.00376806 0.00335242 0.01191556 0.00567523 0.00263673
 0.00426546]
0.8760079694368871
[0.00221854 0.00207506 0.00389749 0.00570577 0.00575888 0.00299898
 0.00387689 0.00455564 0.00274881 0.00360283 0.00265458 0.87337577
 0.00702161 0.00343259 0.00471939 0.00395932 0.00225639 0.00290589
 0.00246658 0.00395663 0.0015